<a href="https://colab.research.google.com/github/quangtrung0220/SieuAmTim/blob/master/SieuAmTim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt

from collections import namedtuple

from sklearn.metrics import classification_report

import cv2

import numpy as np



In [ ]:
# Mount với drive để lấy datasets 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Hàm lấy ra các classes
def get_clases():
  classes = ['2C', '3C', '4C']
  return classes

TrainTest = namedtuple('TrainTest', ['train', 'test'])

# Hàm chuẩn bị dữ liệu
def prepare_data():
  transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomCrop(224, padding=4),
    # transforms.Resize((32,32)),                                    
    # transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
  ])
  transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    # transforms.Resize((32,32)),
    transforms.ToTensor()
  ])
  trainset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/train', transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/test', transform=transform_test)
  return TrainTest(train=trainset, test=testset)

# Hàm load dữ liệu
def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=35, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=35, shuffle=False, num_workers=4)
  return TrainTest(train=trainloader, test=testloader)

In [ ]:
# Data = prepare_data()
# img, label = Data.train[3000]
# print(label)

In [ ]:
# Hàm train 
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  true_result = []
  pred_result = []
  model.train()
  running_loss = 0.0
  reporting_steps = 18
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    true_result += list(labels.cpu().numpy())
    _, predicted = torch.max(outputs, dim=1)
    pred_result += list(predicted.cpu().numpy())
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
  return pred_result, true_result

# hàm test 
def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)

      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

In [ ]:
def main(PATH='./model.pth', model_in=''):
  classes = get_clases()
  datasets = prepare_data()
  # img, label = datasets.train[0]
  # plt.imshow(img)
  # print(classes[label], img.size)
  # print('train', len(datasets.train), 'test', len(datasets.test))
  
  loaders = prepare_loader(datasets)

  # images, labels = iter(loaders.train).next()
  # print(images.shape, labels.shape)

  device = torch.device("cuda:0")
  torch.cuda.empty_cache()

  # model = ResNet50().to(device)
  # model = VGG16().to(device)
  # images, labels = iter(loaders.train).next()
  # outputs = model(images)
  # print(outputs.shape)
  # print(outputs[0])
  # _, predicted = torch.max(outputs, dim=1)
  # print(predicted)
  # imshow(images, labels, predicted, classes)

# Load model 
  if model_in == 'vgg16':
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=3, bias=True)
    if torch.cuda.is_available():
      model.to(device=device)
  elif model_in == 'resnet50':
    model = torchvision.models.resnet50(pretrained=False, progress=False)
    model.fc = torch.nn.modules.linear.Linear(in_features=2048, out_features=3, bias=True)  
    if torch.cuda.is_available():
      model.to(device=device)
  elif model_in == 'desnet':
    model = torchvision.models.densenet121(pretrained=False, progress=False)
    model.classifier = torch.nn.modules.linear.Linear(in_features=1024, out_features=3, bias=True)
    if torch.cuda.is_available():
      model.to(device=device)
  else:
    pass  

  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    print(f'Epoch {epoch} report: ')
    
    pred_result, true_result = train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    print('Train report: ')
    print(classification_report(true_result, pred_result, target_names=classes))

    print('Test report: ')
    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(ytrue, ypred, target_names=classes))

    torch.save(model.state_dict(), PATH)

  return model

model = main(PATH="./resnet50.pth", model_in='resnet50')

In [ ]:
Kết quả sau 10 epoch:

Train_Accuracy:
VGG16_Ảnh resize 224: [35, 38, 38, 41, 54, 70, 82, 89, 93, 95]
VGG16_Ảnh resize 32: [36, 42, 54, 65, 81, 89, 92, 94, 97, 97]
ResNet50_Ảnh resize 224: [35, 38, 38, 41, 54, 70, 82, 89, 93, 95]
Desnet121_Ảnh resize 224: [68, 93, 97, 99, 99, 99, 100, 100, 100, 100]
Desnet121_Ảnh resize 32: [63, 80, 91, 94, 97, 97, 97, 98, 98, 99]

Test_Accuracy
VGG16_Ảnh resize 224: [32, 25, 27, 48, 66, 62, 92, 90, 85, 87]
VGG16_Ảnh resize 32: [23, 56, 63, 74, 82, 83, 87, 85, 91, 83]
ResNet50_Ảnh resize 224: [35, 38, 38, 41, 54, 70, 82, 89, 93, 95]
Desnet121_Ảnh resize 224: [66, 71, 78, 74, 74, 72, 82, 87, 87, 88]
Desnet121_Ảnh resize 32: [78, 83, 79, 78, 77, 72, 70, 78, 78, 82]